# A3
* GCR = 0.4
* Albedo = 0.2
* Hub Height = 3.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* A GCR of 0.4 for the Trina Vertex module (length = 2.384m) corresponds to a rowto-row spacing or pitch of 5.96m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/A3-1'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/A3-1


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+252.gae41aa2.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'A3-1'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 5.96 # m
albedo = 0.2
hub_height = 3.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/A3-1
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-12-01_0000', endtime='2022-12-31_2359')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~240 skyfiles. 
Created 217 skyfiles in /skies/


{'2022-12-01_0730': {'surf_azm': 90.0,
  'surf_tilt': 20.75,
  'theta': -20.75,
  'dni': 596,
  'ghi': 107,
  'dhi': 23,
  'temp_air': 3.2,
  'wind_speed': 0.9,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-12-01_0730.rad'},
 '2022-12-01_0830': {'surf_azm': 90.0,
  'surf_tilt': 55.0,
  'theta': -55.0,
  'dni': 807,
  'ghi': 252,
  'dhi': 34,
  'temp_air': 5.6,
  'wind_speed': 1.0,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-12-01_0830.rad'},
 '2022-12-01_0930': {'surf_azm': 90.0,
  'surf_tilt': 44.71,
  'theta': -44.71,
  'dni': 927,
  'ghi': 414,
  'dhi': 39,
  'temp_air': 7.2,
  'wind_speed': 1.8,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-12-01_0930.rad'},
 '2022-12-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 22.93,
  'theta': -22.93,
  'dni': 984,
  'ghi': 533,
  'dhi': 40,
  'temp_air': 9.5,
  'wind_speed': 2.6,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-12-01_1030.rad'},
 '2022-12-01_1130': {'surf_azm': 270.0,
  'surf_tilt': 2.01,
  'theta': 2.01,
  'dni': 1005,
  'ghi': 593,
  '

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~217 .rad files for gendaylit 1-axis workflow (this takes a minute..)
217 Radfiles created in /objects/

Making 217 octfiles in root directory.
Created 1axis_2022-12-01_0730.oct
Created 1axis_2022-12-01_0830.oct
Created 1axis_2022-12-01_0930.oct
Created 1axis_2022-12-01_1030.oct
Created 1axis_2022-12-01_1130.oct
Created 1axis_2022-12-01_1230.oct
Created 1axis_2022-12-01_1330.oct
Created 1axis_2022-12-01_1430.oct
Created 1axis_2022-12-01_1530.oct
Created 1axis_2022-12-02_0830.oct
Created 1axis_2022-12-02_0930.oct
Created 1axis_2022-12-02_1030.oct
Created 1axis_2022-12-02_1130.oct
Created 1axis_2022-12-02_1230.oct
Created 1axis_2022-12-02_1330.oct
Created 1axis_2022-12-02_1430.oct
Created 1axis_2022-12-03_0830.oct
Created 1axis_2022-12-03_0930.oct
Created 1axis_2022-12-03_1030.oct
Created 1axis_2022-12-03_1130.oct
Created 1axis_2022-12-03_1230.oct
Created 1axis_2022-12-03_1330.oct
Created 1axis_2022-12-03_1430.oct
Created 1axis_2022-12-04_0730.oct
Created 1axis_2022-12-04_0830.oc

In [14]:
trackerdict = demo.analysis1axis(sensorsy=2) # Middle Module

Linescan in process: 1axis_2022-12-01_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-12-01_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-01_0730_Row3_Module13.csv
Index: 2022-12-01_0730. Wm2Front: 293.91225. Wm2Back: 14.605633333333333
Linescan in process: 1axis_2022-12-01_0830_Row3_Module13_Front
Linescan in process: 1axis_2022-12-01_0830_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-01_0830_Row3_Module13.csv
Index: 2022-12-01_0830. Wm2Front: 620.3073833333333. Wm2Back: 15.868631666666666
Linescan in process: 1axis_2022-12-01_0930_Row3_Module13_Front
Linescan in process: 1axis_2022-12-01_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-01_0930_Row3_Module13.csv
Index: 2022-12-01_0930. Wm2Front: 635.8702666666666. Wm2Back: 44.41810666666667
Linescan in process: 1axis_2022-12-01_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-12-01_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-01_1030_Row3_Module13.csv
Index: 2022-12-01_1030. Wm2Fro

Linescan in process: 1axis_2022-12-04_1530_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-04_1530_Row3_Module13.csv
Index: 2022-12-04_1530. Wm2Front: 65.95324500000001. Wm2Back: 8.111126833333334
Linescan in process: 1axis_2022-12-05_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-12-05_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-05_0730_Row3_Module13.csv
Index: 2022-12-05_0730. Wm2Front: 76.67904. Wm2Back: 7.548118166666668
Linescan in process: 1axis_2022-12-05_0930_Row3_Module13_Front
Linescan in process: 1axis_2022-12-05_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-05_0930_Row3_Module13.csv
Index: 2022-12-05_0930. Wm2Front: 176.72781666666668. Wm2Back: 33.303403333333335
Linescan in process: 1axis_2022-12-05_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-12-05_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-05_1030_Row3_Module13.csv
Index: 2022-12-05_1030. Wm2Front: 224.04478333333336. Wm2Back: 30.723066666666668
Linescan in 

Linescan in process: 1axis_2022-12-11_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-11_0930_Row3_Module13.csv
Index: 2022-12-11_0930. Wm2Front: 629.5137833333333. Wm2Back: 40.35607833333333
Linescan in process: 1axis_2022-12-11_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-12-11_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-11_1030_Row3_Module13.csv
Index: 2022-12-11_1030. Wm2Front: 593.59155. Wm2Back: 61.156675
Linescan in process: 1axis_2022-12-11_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-12-11_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-11_1130_Row3_Module13.csv
Index: 2022-12-11_1130. Wm2Front: 575.6567. Wm2Back: 65.52940833333332
Linescan in process: 1axis_2022-12-11_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-12-11_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-11_1230_Row3_Module13.csv
Index: 2022-12-11_1230. Wm2Front: 609.6976. Wm2Back: 61.75817666666666
Linescan in process: 1axis_2022-12-11_133

Linescan in process: 1axis_2022-12-14_1530_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-14_1530_Row3_Module13.csv
Index: 2022-12-14_1530. Wm2Front: 335.07503333333335. Wm2Back: 23.500181666666663
Linescan in process: 1axis_2022-12-15_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-12-15_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-15_0730_Row3_Module13.csv
Index: 2022-12-15_0730. Wm2Front: 226.45344999999998. Wm2Back: 16.260063333333335
Linescan in process: 1axis_2022-12-15_0830_Row3_Module13_Front
Linescan in process: 1axis_2022-12-15_0830_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-15_0830_Row3_Module13.csv
Index: 2022-12-15_0830. Wm2Front: 559.4583. Wm2Back: 15.976519999999999
Linescan in process: 1axis_2022-12-15_0930_Row3_Module13_Front
Linescan in process: 1axis_2022-12-15_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-15_0930_Row3_Module13.csv
Index: 2022-12-15_0930. Wm2Front: 604.9977833333332. Wm2Back: 41.66820333333334
Linescan in

Linescan in process: 1axis_2022-12-18_1330_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-18_1330_Row3_Module13.csv
Index: 2022-12-18_1330. Wm2Front: 302.32803333333334. Wm2Back: 45.88192666666667
Linescan in process: 1axis_2022-12-18_1430_Row3_Module13_Front
Linescan in process: 1axis_2022-12-18_1430_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-18_1430_Row3_Module13.csv
Index: 2022-12-18_1430. Wm2Front: 255.02251666666666. Wm2Back: 38.043705
Linescan in process: 1axis_2022-12-18_1530_Row3_Module13_Front
Linescan in process: 1axis_2022-12-18_1530_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-18_1530_Row3_Module13.csv
Index: 2022-12-18_1530. Wm2Front: 141.7205166666667. Wm2Back: 14.336518333333332
Linescan in process: 1axis_2022-12-19_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-12-19_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-19_0730_Row3_Module13.csv
Index: 2022-12-19_0730. Wm2Front: 237.1322. Wm2Back: 16.884565000000002
Linescan in process:

Linescan in process: 1axis_2022-12-22_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-22_0930_Row3_Module13.csv
Index: 2022-12-22_0930. Wm2Front: 630.0675666666666. Wm2Back: 41.124181666666665
Linescan in process: 1axis_2022-12-22_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-12-22_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-22_1030_Row3_Module13.csv
Index: 2022-12-22_1030. Wm2Front: 595.1563166666667. Wm2Back: 59.815628333333336
Linescan in process: 1axis_2022-12-22_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-12-22_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-22_1130_Row3_Module13.csv
Index: 2022-12-22_1130. Wm2Front: 545.38275. Wm2Back: 61.801565000000004
Linescan in process: 1axis_2022-12-22_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-12-22_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-22_1230_Row3_Module13.csv
Index: 2022-12-22_1230. Wm2Front: 591.0441666666666. Wm2Back: 61.7042
Linescan in process: 1

Linescan in process: 1axis_2022-12-25_1530_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-25_1530_Row3_Module13.csv
Index: 2022-12-25_1530. Wm2Front: 309.05348333333336. Wm2Back: 20.252685
Linescan in process: 1axis_2022-12-26_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-12-26_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-26_0730_Row3_Module13.csv
Index: 2022-12-26_0730. Wm2Front: 227.05518333333333. Wm2Back: 16.927343333333333
Linescan in process: 1axis_2022-12-26_0830_Row3_Module13_Front
Linescan in process: 1axis_2022-12-26_0830_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-26_0830_Row3_Module13.csv
Index: 2022-12-26_0830. Wm2Front: 554.2853. Wm2Back: 13.44433
Linescan in process: 1axis_2022-12-26_0930_Row3_Module13_Front
Linescan in process: 1axis_2022-12-26_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-12-26_0930_Row3_Module13.csv
Index: 2022-12-26_0930. Wm2Front: 573.5862666666667. Wm2Back: 39.83336833333334
Linescan in process: 1axis_202

In [15]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

Linescan in process: 1axis_2022-12-01_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-12-01_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-01_0730_Row3_Module1.csv
Index: 2022-12-01_0730. Wm2Front: 296.11525. Wm2Back: 24.465008333333333
Linescan in process: 1axis_2022-12-01_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-12-01_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-01_0830_Row3_Module1.csv
Index: 2022-12-01_0830. Wm2Front: 631.5038333333334. Wm2Back: 49.779399999999995
Linescan in process: 1axis_2022-12-01_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-12-01_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-01_0930_Row3_Module1.csv
Index: 2022-12-01_0930. Wm2Front: 643.3338666666667. Wm2Back: 76.70484333333334
Linescan in process: 1axis_2022-12-01_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-12-01_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-01_1030_Row3_Module1.csv
Index: 2022-12-01_1030. Wm2Front: 604.1313

Linescan in process: 1axis_2022-12-05_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-05_0730_Row3_Module1.csv
Index: 2022-12-05_0730. Wm2Front: 78.67109333333335. Wm2Back: 10.238118166666666
Linescan in process: 1axis_2022-12-05_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-12-05_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-05_0930_Row3_Module1.csv
Index: 2022-12-05_0930. Wm2Front: 182.77171666666663. Wm2Back: 42.09901166666667
Linescan in process: 1axis_2022-12-05_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-12-05_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-05_1030_Row3_Module1.csv
Index: 2022-12-05_1030. Wm2Front: 225.98641666666668. Wm2Back: 39.51031666666667
Linescan in process: 1axis_2022-12-05_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-12-05_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-05_1230_Row3_Module1.csv
Index: 2022-12-05_1230. Wm2Front: 230.36066666666665. Wm2Back: 40.976128333333335
Linescan in p

Linescan in process: 1axis_2022-12-11_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-11_1030_Row3_Module1.csv
Index: 2022-12-11_1030. Wm2Front: 595.5490166666666. Wm2Back: 98.15448333333333
Linescan in process: 1axis_2022-12-11_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-12-11_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-11_1130_Row3_Module1.csv
Index: 2022-12-11_1130. Wm2Front: 575.76935. Wm2Back: 108.53696666666667
Linescan in process: 1axis_2022-12-11_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-12-11_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-11_1230_Row3_Module1.csv
Index: 2022-12-11_1230. Wm2Front: 611.4418166666667. Wm2Back: 100.13193333333334
Linescan in process: 1axis_2022-12-11_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-12-11_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-11_1330_Row3_Module1.csv
Index: 2022-12-11_1330. Wm2Front: 679.9160333333333. Wm2Back: 76.04295833333333
Linescan in process: 1ax

Linescan in process: 1axis_2022-12-15_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-15_0730_Row3_Module1.csv
Index: 2022-12-15_0730. Wm2Front: 228.03278333333333. Wm2Back: 19.949593333333333
Linescan in process: 1axis_2022-12-15_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-12-15_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-15_0830_Row3_Module1.csv
Index: 2022-12-15_0830. Wm2Front: 573.4549999999999. Wm2Back: 46.11472499999999
Linescan in process: 1axis_2022-12-15_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-12-15_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-15_0930_Row3_Module1.csv
Index: 2022-12-15_0930. Wm2Front: 615.9951666666666. Wm2Back: 73.18688666666668
Linescan in process: 1axis_2022-12-15_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-12-15_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-15_1030_Row3_Module1.csv
Index: 2022-12-15_1030. Wm2Front: 580.4032. Wm2Back: 95.72922166666667
Linescan in process: 1axi

Linescan in process: 1axis_2022-12-18_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-18_1430_Row3_Module1.csv
Index: 2022-12-18_1430. Wm2Front: 274.67715. Wm2Back: 51.673064999999994
Linescan in process: 1axis_2022-12-18_1530_Row3_Module1_Front
Linescan in process: 1axis_2022-12-18_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-18_1530_Row3_Module1.csv
Index: 2022-12-18_1530. Wm2Front: 143.87481666666667. Wm2Back: 19.04271
Linescan in process: 1axis_2022-12-19_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-12-19_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-19_0730_Row3_Module1.csv
Index: 2022-12-19_0730. Wm2Front: 239.00500000000002. Wm2Back: 20.318618333333333
Linescan in process: 1axis_2022-12-19_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-12-19_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-19_0830_Row3_Module1.csv
Index: 2022-12-19_0830. Wm2Front: 592.6347000000001. Wm2Back: 43.85830166666667
Linescan in process: 1axis_2022

Linescan in process: 1axis_2022-12-22_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-22_1030_Row3_Module1.csv
Index: 2022-12-22_1030. Wm2Front: 598.6127666666666. Wm2Back: 97.12368666666666
Linescan in process: 1axis_2022-12-22_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-12-22_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-22_1130_Row3_Module1.csv
Index: 2022-12-22_1130. Wm2Front: 545.1954499999999. Wm2Back: 102.46376666666667
Linescan in process: 1axis_2022-12-22_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-12-22_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-22_1230_Row3_Module1.csv
Index: 2022-12-22_1230. Wm2Front: 593.9490166666667. Wm2Back: 100.00688166666666
Linescan in process: 1axis_2022-12-22_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-12-22_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-22_1330_Row3_Module1.csv
Index: 2022-12-22_1330. Wm2Front: 666.1985333333333. Wm2Back: 81.93388999999999
Linescan in proc

Linescan in process: 1axis_2022-12-26_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-26_0730_Row3_Module1.csv
Index: 2022-12-26_0730. Wm2Front: 228.4769333333333. Wm2Back: 19.772090000000002
Linescan in process: 1axis_2022-12-26_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-12-26_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-26_0830_Row3_Module1.csv
Index: 2022-12-26_0830. Wm2Front: 567.1519499999999. Wm2Back: 43.270669999999996
Linescan in process: 1axis_2022-12-26_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-12-26_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-26_0930_Row3_Module1.csv
Index: 2022-12-26_0930. Wm2Front: 586.2797333333333. Wm2Back: 69.47642166666667
Linescan in process: 1axis_2022-12-26_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-12-26_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-12-26_1030_Row3_Module1.csv
Index: 2022-12-26_1030. Wm2Front: 568.2793166666667. Wm2Back: 91.90956833333333
Linescan in proc

In [16]:
#trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

In [17]:
demo.calculateResults()

Bifaciality factor of module stored is  1
No CECModule data passed; using default for Prism Solar BHC72-400


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

{'2022-12-01_0730': {'surf_azm': 90.0,
  'surf_tilt': 20.75,
  'theta': -20.75,
  'dni': 596,
  'ghi': 107,
  'dhi': 23,
  'temp_air': 3.2,
  'wind_speed': 0.9,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-12-01_0730.rad',
  'radfile': 'objects/1axis2022-12-01_0730__C_3.11134_rtr_5.96000_tilt_20.75000_25modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 1.303, 'y': 2.384, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.305, 'sceney': 2.384, 'scenez': 0.095, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 1.303 2.384 0.02 | xform -t -0.6515 -1.192 0.095 -a 1 -t 0 2.384 0\r\n! genrev Metal_Grey tube1 t*1.305 0.075 32 | xform -ry 90 -t -0.6525 0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0.095, 'xgap': 0.002, 'ygap': 0.0, 'zgap': 0.02}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.39999999999999997, 'text': '!xform -rx 20.75 -t 0 0 3.5 -a 25 -t 1.305 0 0 -a 5 -t 0 5.96 0 -i 1 -t -15.66 -11.92 0 -rz 90.0 -t 0 0 0 ob

In [18]:
demo.CompiledResults.set_index('timestamp', inplace=True)
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

,module,row,POA_eff,Grear_mean,Gfront_mean,Module_temp,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
timestamp,,,,,,,,,,,,,,,,
2022-12-01_0730,13,3,308.517883,14.605633,293.912250,12.326748,128.893628,122.860850,4.969386,4.910252,0.000169,128.893410,0.9,596,23,107
2022-12-01_0730,1,3,320.580258,24.465008,296.115250,12.683584,133.861920,123.771958,8.261989,8.152058,0.000002,133.861918,0.9,596,23,107
2022-12-01_0830,13,3,636.176015,15.868632,620.307383,24.293354,256.566423,250.593917,2.558188,2.383340,0.000117,256.566122,1.0,807,34,252
2022-12-01_0830,1,3,681.283233,49.779400,631.503833,25.618782,273.388301,254.810881,7.882676,7.290670,0.000086,273.388066,1.0,807,34,252
2022-12-01_0930,13,3,680.288373,44.418107,635.870267,26.144299,272.465112,255.868203,6.985404,6.486507,0.000114,272.464801,1.8,927,39,414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28_1530,1,3,90.565348,12.421997,78.143352,10.138773,36.805292,31.621132,15.896422,16.394604,0.000291,36.805185,5.8,1,76,81
2022-12-29_0830,13,3,581.037337,11.834703,569.202633,18.932088,238.898933,234.305671,2.079172,1.960371,0.000135,238.898611,1.3,741,33,204
2022-12-29_0830,1,3,621.833310,41.878060,579.955250,20.106887,254.615381,238.479587,7.220912,6.766111,0.000052,254.615248,1.3,741,33,204


In [19]:
# demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'A3-Jan.csv')

In [20]:
demo.CompiledResults.to_csv(Path().resolve().parent.parent / 'Results' / 'A3-all-Dec.csv')